In [92]:
from keras import Sequential
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD
from keras.utils import to_categorical
from numpy import mean, std
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt

In [93]:
# normalises data to fit [0,1]
def normalise(data):
    return data.astype('float') / 255.0

In [94]:
def load_mnist():
    # load mnist dataset
    (train_x, train_y), (test_x, test_y) = mnist.load_data()

    # reshape pixel-data arrays to contain only one colour channel
    train_x = train_x.reshape((train_x.shape[0], 28, 28, 1))
    test_x = test_x.reshape((test_x.shape[0], 28, 28, 1))

    # normalise pixel-data
    train_x = normalise(train_x)
    test_y = normalise(train_y)

    # transform labels to 10 dimension indicator vector
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)

    return train_x, train_y, test_x, test_y

In [95]:
def define_basic_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3),
                     activation='relu',
                     kernel_initializer='he_uniform',
                     input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(100,
                    activation='relu',
                    kernel_initializer='he_uniform'))
    model.add(Dense(10,
                    activation='softmax'))
    optimizer = SGD(learning_rate=.01, momentum=.9, )

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
def evaluate_model(dataX, dataY, n_folds=5):
     scores, histories = list(), list()
     # prepare cross validation
     kfold = KFold(n_folds, shuffle=True, random_state=1)
     # enumerate splits
     for train_ix, test_ix in kfold.split(dataX):
         model = define_basic_model()
         # select rows for train and test
         trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
         # fit model
         history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
         # evaluate model
         _, acc = model.evaluate(testX, testY, verbose=0)
         print('> %.3f' % (acc * 100.0))
         # stores scores
         scores.append(acc)
         histories.append(history)
     return scores, histories

In [101]:
def kfold_cross_validation(model, train_x, train_y, test_x, test_y):
    # store histories (records of training losses during training) and accuracies (records of accuracy post training)
    accuracies, histories = list(), list()

    # split data into 5-fold cross-validation
    kfold = KFold(5, shuffle=True, random_state=0)

    for train_index, validation_index in kfold.split(train_x):
        # split training data into respective train/validation folds
        train_x_fold, train_y_fold = train_x[train_index], train_y[train_index]
        validation_x_fold, validation_y_fold = train_x[validation_index], train_y[validation_index]

        # train model and store history
        history = model.fit(train_x_fold, train_y_fold,
                            epochs=10,
                            batchsize=32,
                            validation_data=(validation_x_fold, validation_y_fold))
        histories.append(history)

        # evaluate model (ignore logs)
        _, accuracy = model.evaluate(test_x, test_y)
        accuracies.append(accuracy)
        print(accuracy)
    return accuracy, histories

In [97]:
def summarize_diagnostics(histories):
 for i in range(len(histories)):
     # plot loss
     plt.subplot(2, 1, 1)
     plt.title('Cross Entropy Loss')
     plt.plot(histories[i].history['loss'], color='blue', label='train')
     plt.plot(histories[i].history['val_loss'], color='orange', label='test')
     # plot accuracy
     plt.subplot(2, 1, 2)
     plt.title('Classification Accuracy')
     plt.plot(histories[i].history['accuracy'], color='blue', label='train')
     plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
     plt.show()

In [98]:
def summarize_performance(scores):
 # print summary
 print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
 # box and whisker plots of results
 plt.boxplot(scores)
 plt.show()

In [99]:
def run_test_harness():
 # load dataset
 trainX, trainY, testX, testY = load_mnist()
 # prepare pixel data
 trainX, testX = normalise(trainX), normalise(trainY)
 # evaluate model
 model = define_basic_model()
 scores, histories = kfold_cross_validation(model, trainX, trainY, testX, testY)
 # learning curves
 summarize_diagnostics(histories)
 # summarize estimated performance
 summarize_performance(scores)

In [ ]:
run_test_harness()